NDB Model

**Sources**:
- [Illustrating the Performance of the NBD as a Benchmark Model for Customer-Base Analysis](http://www.brucehardie.com/notes/005/)

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import beta, gamma, gammaln, factorial, hyp2f1
from scipy.stats import chisquare, chi2

import matplotlib.pyplot as plt
import matplotlib_inline
from IPython.display import display_markdown

from utils import CDNOW, bic

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False

In [2]:
data = (
    CDNOW(master=False, calib_p=273)
    .rfm_summary()
    .select('P1X', 't_x', 'T')
)

# x: 'repeat_trans' = “repeat frequency”, number of repeat transactions made by a customer in a specified time period.
# t_x: 'last_purch' (in weeks) = Time of last calibration period repeat purchase 
# T: 'T' (in weeks) = length of time over which we have had an opportunity to observe any repeat purchasing behavior. 
# 'first_purch' = Time of first purchase (in weeks)
repeat_trans, last_purch, T = np.hsplit(data.collect().to_numpy(), 3)
first_purch = 39 - T
num_customers = len(repeat_trans)

In [3]:
# NBD Model - timing-model equivalent of the basic NBD model
def nbd_timing_params(x, T):
    def log_likelihood(params):
        r, alpha = params
        return -np.sum(gammaln(r+x)-gammaln(r)+r*np.log(alpha)-(r+x)*np.log(alpha+T))
    return minimize(log_likelihood, x0=[0.1,0.1], bounds=[(1e-6, np.inf), (1e-6, np.inf)])

# NBD Model - standard counting form
def nbd_params(x, T):
    def log_likelihood(params):
        r, alpha = params
        # P(X(T)=x)
        pmf = np.exp(gammaln(r+x)-gammaln(r))/factorial(x) * (alpha/(alpha+T))**r * (T/(alpha+T))**x
        return -np.sum(np.log(pmf))
    return minimize(log_likelihood, x0=[0.1,0.1], bounds=[(1e-6, np.inf), (1e-6, np.inf)])

# BG/NBD Model
def bgnbd_params(x, t_x, T):
    def log_likelihood(params):
        r, alpha, a, b = params
        likelihood_function = (
            beta(a,b+x)/beta(a,b) * 
            (gamma(r+x)*alpha**r)/(gamma(r)*(alpha+T)**(r+x))
        )
        likelihood_function += (
            np.where(x > 0,
                beta(a+1, b+x-1)/beta(a,b) *
                (gamma(r+x)*alpha**r)/(gamma(r)*(alpha+t_x)**(r+x)),
                0)
        )
        return -np.sum(np.log(likelihood_function))
    
    bnds = [(1e-6, np.inf) for _ in range(4)]
    guess = [0.01 for _ in range(4)]
    return minimize(log_likelihood, x0=guess, bounds=bnds)

# Pareto/NBD Model
def paretonbd_params(x, t_x, T):
    
    def log_likelihood(params):
        r, alpha, s, beta = params
        
        maxab = np.max((alpha, beta))
        absab = np.abs(alpha - beta)
        param2 = s + 1
        if alpha < beta:
            param2 = r + x
            
        part1 = (alpha**r * beta**s / gamma(r)) * gamma(r+x)
        part2 = 1/((alpha+T)**(r+x) * (beta+T)**s)

        if absab == 0:
            F1 = 1/((maxab+t_x)**(r+s+x))
            F2 = 1/((maxab+T)**(r+s+x))
        else:
            F1 = hyp2f1(r+s+x, param2, r+s+x+1, absab/(maxab+t_x)) / ((maxab+t_x)**(r+s+x))
            F2 = hyp2f1(r+s+x, param2, r+s+x+1, absab/(maxab+T)) / ((maxab+T)**(r+s+x))        
        
        return -np.sum(np.log(part1*(part2+(s/(r+s+x))*(F1-F2))))
    
    bnds = [(1e-6, 20) for _ in range(4)]
    guess = [0.01 for _ in range(4)]
    return minimize(log_likelihood, x0=guess, bounds=bnds)

In [4]:
res = nbd_timing_params(repeat_trans, T)
r, alpha = res.x
ll = res.fun

display_markdown(f'''**NBD - Timing-Model Equivalent:**

Parameter Estimates:
- $r$ = {r:0.4f}
- $\\alpha$ = {alpha:0.4f}

Log-Likelihood = {-ll:0.4f}

BIC = {bic(2, num_customers, ll):.1f}''', raw=True)

res = nbd_params(repeat_trans, T)
r, alpha = res.x
ll = res.fun

display_markdown(f'''**NBD - Standard Counting Method:**

Parameter Estimates:
- $r$ = {r:0.4f}
- $\\alpha$ = {alpha:0.4f}

Log-Likelihood = {-ll:0.4f}

BIC = {bic(2, num_customers, ll):.1f}''', raw=True)

res = bgnbd_params(repeat_trans, last_purch, T)
r, alpha, a, b = res.x
ll = res.fun

display_markdown(f'''**BG/NBD:**

Parameter Estimates:
- $r$ = {r:0.4f}
- $\\alpha$ = {alpha:0.4f}
- $a$ = {a:0.4f}
- $b$ = {b:0.4f}

Log-Likelihood = {-ll:0.4f}

BIC = {bic(4, num_customers, ll):.1f}''', raw=True)

res = paretonbd_params(repeat_trans, last_purch, T)
r, alpha, s, beta_param = res.x
ll = res.fun

display_markdown(f'''**Pareto/NBD:**

Parameter Estimates:
- $r$ = {r:0.4f}
- $\\alpha$ = {alpha:0.4f}
- $s$ = {s:0.4f}
- $\\beta$ = {beta_param:0.4f}

Log-Likelihood = {-ll:0.4f}

BIC = {bic(4, num_customers, ll):.1f}''', raw=True)

**NBD - Timing-Model Equivalent:**

Parameter Estimates:
- $r$ = 0.3848
- $\alpha$ = 12.0719

Log-Likelihood = -9763.6576

BIC = 19542.8

**NBD - Standard Counting Method:**

Parameter Estimates:
- $r$ = 0.3848
- $\alpha$ = 12.0718

Log-Likelihood = -3193.0587

BIC = 6401.6

**BG/NBD:**

Parameter Estimates:
- $r$ = 0.2426
- $\alpha$ = 4.4136
- $a$ = 0.7929
- $b$ = 2.4259

Log-Likelihood = -9582.4292

BIC = 19195.9

**Pareto/NBD:**

Parameter Estimates:
- $r$ = 0.5533
- $\alpha$ = 10.5778
- $s$ = 0.6063
- $\beta$ = 11.6661

Log-Likelihood = -9594.9762

BIC = 19221.0

Predicted Distribution of Transactions

In [5]:
num_repeats, repeat_freq = np.unique(repeat_trans, return_counts=True)

# right-censored distribution in which counts greater than 7 are collapsed into a 7+ bin
num_repeats_censored = num_repeats[:8]
repeat_freq_censored = repeat_freq[:8]
repeat_freq_censored[-1] = np.sum(repeat_freq) - np.sum(repeat_freq[:7])

first_purch_uq, first_purch_count = np.unique(first_purch, return_counts=True)

first_purch_count

array([18, 22, 17, 20, 23, 25, 32, 18, 24, 18, 25, 26, 28, 25, 22, 22, 22,
       27, 29, 35, 25, 25, 29, 26, 27, 26, 29, 31, 33, 30, 22, 29, 30, 36,
       36, 25, 43, 26, 31, 29, 33, 33, 25, 25, 25, 22, 35, 36, 34, 30, 25,
       25, 32, 24, 37, 31, 36, 34, 30, 26, 31, 30, 32, 40, 29, 29, 28, 31,
       30, 30, 26, 28, 32, 25, 34, 28, 31, 28, 21, 19, 24, 24, 33, 30])